In [ ]:
import torch
import os
import cv2
import torchvision.models as models
import torchvision.transforms as transforms
from tqdm import tqdm

# Initialize the pre-trained model (e.g., ResNet18)
model = models.resnet18(pretrained=True)
model.eval()

# Define a transform to resize and normalize video frames
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to extract video features using the pre-trained model
def extract_video_features(video_path):
    cap = cv2.VideoCapture(video_path)
    features = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Preprocess the frame
        input_tensor = preprocess(frame)
        input_tensor = input_tensor.unsqueeze(0)  # Add batch dimension

        # Extract features
        with torch.no_grad():
            feature = model(input_tensor)
        
        features.append(feature.squeeze(0))  # Remove batch dimension
    cap.release()
    return features

# Function to pad frames to the maximum length
def pad_frames(frames, max_length, padding_value=0):
    if not frames:
        # If frames is empty, return a tensor of zeros with the correct shape
        return torch.full((max_length, 512), padding_value)
    
    num_frames = len(frames)
    if num_frames < max_length:
        padding_frames = [torch.full_like(frames[0], padding_value) for _ in range(max_length - num_frames)]
        frames.extend(padding_frames)
    return torch.stack(frames)

# Example usage: video file paths
video_dir = r'C:\Users\T 480\MALD\MELD\train\train_splits'
video_paths = [os.path.join(video_dir, f) for f in os.listdir(video_dir) if f.endswith('.mp4')]

# Extract features for all videos along with their filenames
all_video_features = []
for video_path in tqdm(video_paths):
    video_features = extract_video_features(video_path)
    video_filename = os.path.basename(video_path)
    all_video_features.append((video_filename, video_features))

# Determine maximum number of frames
max_frames = max(len(video) for _, video in all_video_features)

# Pad all video features to the same length and store filenames with padded features
all_padded_video_features = [(filename, pad_frames(video, max_frames)) for filename, video in all_video_features]

# Save the filenames and their corresponding padded features
torch.save(all_padded_video_features, 'filenames_and_padded_video_features.pt')
